In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams.update({'figure.figsize':(10,6), 'image.cmap':'RdBu_r',
                     'font.size':14})

In [2]:
import datetime

In [3]:
import stravalib

In [4]:
with open('../api_key') as api_file:
    CLIENT_ID = api_file.readline().replace('\n','')
    CLIENT_SECRET = api_file.readline().strip()
    ACCESS_TOKEN = api_file.readline().strip()

In [5]:
# access using stored access token 
# can use id + secret to retrieve token as well using 
# client.exchange_code_for_token
client = stravalib.Client(access_token=ACCESS_TOKEN) 

print(client.get_athlete()) # me 

print(client.get_athlete(athlete_id=7656735))#stephanie

<Athlete id=1639875 firstname=b'Jake' lastname=b'Edman'>
<Athlete id=7656735 firstname=b'Stephanie' lastname=b'Wuerth'>


In [6]:
# get segment ids from most recent runs
segment_ids = []
for a in client.get_activities( limit = 5 ):
    # this is jank but I can't pass include_all_efforts to get_activities
    activity = client.get_activity(a.id, include_all_efforts = True) # return all segment efforts
    for effort in activity.segment_efforts:
        segment_ids.append(effort.segment.id)

In [7]:
print('number of segment efforts: {}'.format( len(segment_ids)))
segment_ids_unique = list(set(segment_ids))
print('number of unique segments: {}'.format( len(segment_ids_unique)))
# could choose to weight by frequency of segments, or not 

number of segment efforts: 64
number of unique segments: 60


In [34]:
# fixed stravalib so it takes context_entries now
# can return the whole leaderboard by requesting top_results_limit = 0, but that seems bad
test = client.get_segment_leaderboard(segment_id = segment_ids[1], top_results_limit=1, context_entries=5)
# still can't get it to print less than 1 of the top entries though-- will have to strip that off 

In [86]:
## get a dictionary with segment_id as key, with all the nearby athletes in a dict 
segment_rivals = dict()
counter = 0
total = len(segment_ids_unique)
athlete_names = dict() # master table

for segment_id in segment_ids_unique[0:10]:
    counter = counter + 1
    try:
        leaderboard = client.get_segment_leaderboard(segment_id = segment_id,
                                                     top_results_limit=1, context_entries=3)
        print('on segment {}, {} of {}'.format(segment_id, counter, total ))
    except:
        # janky error handling, should specify http error -- probably need urllib2
        print('whoops, error on {}, {} of {}'.format(segment_id, counter, total ))
    
    segment_athletes = dict()
    for i, entry in enumerate(leaderboard):
        # remove first entry 
        # should only do this if athlete isn't in top ranking
        if i == 0:
            continue
        else:
            segment_athletes[entry.athlete_id] = entry.athlete_name
            athlete_names[entry.athlete_id] = entry.athlete_name
    
    segment_rivals[segment_id] = segment_athletes
    
            
        
        
    

on segment 6390656, 1 of 60
on segment 10239747, 2 of 60
on segment 2548356, 3 of 60
on segment 2773381, 4 of 60
on segment 1679751, 5 of 60
on segment 780296, 6 of 60
on segment 11450121, 7 of 60
on segment 780298, 8 of 60
on segment 775819, 9 of 60
on segment 1710600, 10 of 60


In [87]:
segment_rivals

{775819: {206866: 'Kilo Papa',
  1061806: 'David Johnson',
  1287436: 'Annie Lydens',
  1639875: 'Jake Edman',
  2149310: 'rstever mv',
  2673746: 'Hallie Fox',
  12234678: 'Thorsten Preiss'},
 780296: {982877: 'burke green',
  1382509: 'Michael Beetham',
  1639875: 'Jake Edman',
  1727743: 'Topher Gaylord',
  3843119: 'Garret Christensen',
  4064133: 'Travis Weller',
  7347523: 'Hanae Rivera'},
 780298: {63664: 'Yoni Doron-Peters',
  356573: 'Ken Brunt',
  417280: 'Dominick Layfield',
  1134749: 'Matthew Schmidt',
  1639875: 'Jake Edman',
  5928301: 'Ethan Salter',
  10238576: 'Todd Watson'},
 1679751: {276150: 'Mara Guillemette Snipes',
  583086: 'Zach Krause',
  1095274: 'Mark Tanaka',
  1205982: 'AJ Cantu',
  1639875: 'Jake Edman',
  4345638: 'Spenser T',
  4747576: 'Ted McCarthy'},
 1710600: {3379: 'Erik Wilde',
  1639875: 'Jake Edman',
  1785499: 'Byron Pittam',
  6074832: 'Jonathan King',
  7099795: 'Louie Cao',
  10638417: 'James Riley',
  13717077: 'Colin Gannon'},
 2548356: {

In [150]:
# make a frame with all of the rivals
df = pd.DataFrame.from_dict(segment_rivals, orient = 'index')
athlete_counts = df.count().to_frame('counts')


In [159]:
# make a frame with the names and ids
names = pd.Series(athlete_names).to_frame('athlete_name')
names.rename(index = {0:'athlete_id'}, inplace=True)

In [171]:
# join the frames
rival_counts = pd.merge(athlete_counts, names, how = 'inner', left_index = True, right_index = True)
rival_counts.index.name = 'athlete_id'

In [172]:
# here's the result! luckily i'm my own most common competitor
rival_counts.sort_values('counts', ascending=False)

,counts,athlete_name
athlete_id,,
1639875,10,Jake Edman
7325,2,Mike Sweeney
3843119,2,Garret Christensen
1090016,2,Eric Dasmalchi
3379,1,Erik Wilde
4975742,1,Max Podell
2149310,1,rstever mv
2162636,1,Jake Tennant
2202121,1,Chris Wheeler
